In [172]:
import requests
import pandas as pd
import random

from bs4 import BeautifulSoup
from selenium import webdriver

In [173]:
def scrape(url):

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 OPR/94.0.0.0'})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [174]:
def add_attribute(attribute):
    if attribute:
        return attribute.text.strip()
    return None

In [175]:
def convert_to_list(element):
    list = []

    for element in element:
        if element:
            return list.append(element.text.strip().replace('\n', ' change here'))
        return list.append(None)

In [176]:
def get_product(url):
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    html = driver.page_source

    soup = BeautifulSoup(html, features="html.parser")

    # products = soup.find('div', class_='list__container')

    attributes = {'Name':[], 'Image Link':[], 'Origin':[], 'Price':[], 'Unit Price':[], 'Bio':[]}
    
    attributes['Name'] = convert_to_list(soup.find_all('p', class_='product-thumbnail__description'))
    attributes['Price'] = convert_to_list(soup.find_all('div', class_='product-price bolder text-dark-color'))    
    attributes['Image Link'] = convert_to_list(soup.find_all('img', class_='product-thumbnail__image'))
    attributes['Origin'] = convert_to_list(soup.find_all('span', class_='rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo'))
    attributes['Unit Price'] = convert_to_list(soup.find_all('span', class_='text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy'))
    

    #for product in products:
        #product.find('p', class_='product-thumbnail__description', itemprop='name description')

    # for product in products:
    #     name = product.find('p', class_='product-thumbnail__description')
    #     origin = product.find('span', class_='rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo')
    #     price = product.find('div', class_='product-price bolder text-dark-color')
    #     #unit_price = product.find('span', class_='text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy')

    #     attributes['Name'].append(add_attribute(name))
    #     attributes['Origin'].append(add_attribute(origin))
    #     attributes['Price'].append(add_attribute(price))
    #     attributes['Unit Price'].append(None)
    #     attributes['Image Link'].append(None)

    for name in attributes['Name']:
        if name:
            if 'bio' in name.lower():
                attributes['Bio'].append(True)
            else:
                attributes['Bio'].append(False)
        else:
            attributes['Bio'].append(None)

    for items in attributes:
        print(len(attributes[items]))

    attribute_df = pd.DataFrame(attributes) 
    attribute_df.dropna(how = 'all', inplace=True)

    return attribute_df

In [177]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [178]:
page_counter = 1
url = 'https://www.auchan.fr/fruits-legumes/fruits/ca-n0301?page=1'

while page_counter <= get_max_page(url):
    retail_df = get_product(url[:-1] + str(page_counter))
    page_counter += 1
    break

retail_df

None


TypeError: 'NoneType' object is not iterable

In [112]:
#retail_df.to_csv('auchan.csv', index=False)